In [12]:
# !pip install langchain
# !pip install langchain-openai
# !pip install tiktoken

In [13]:
import os

# # Set OPENAI API Key

# os.environ["OPENAI_API_KEY"] = "your openai key"

# OR (load from .env file)

# from dotenv import load_dotenv
# load_dotenv("./.env")

The more complex version of this would use this pygithub package as shown in here:
- https://python.langchain.com/docs/integrations/toolkits/github

But let's try a simpler version that just sets up a bunch of python functions to automate different github actions like add, commit and pull requests.

Let's start with a simple commit to a branch of some repository.

First, I'll create a github repository using some simple commands.

In [14]:
# git init -b main

# git add . && git commit -m "Some commit"

# gh repo create

Now, let's try asking an agent to write come code and commit the resulting code to the current branch of this repository.

To do that, let's give the agent the necessary tools it will need which in this case will be python functions that perform different github actions using the `subprocess` package.

We'll follow the basic steps for building a langchain agent:

# Steps for building a simple agent:
- Set up the LLM
- Define the tool or toolkit (list of tools)
- Set up a prompt template
- Connect llm with your tools
- Define your agent as a dict with keys: input, and agent scratchpad 
- Use the Langchain LCEL language to pipe agent, prompt the llm_with_tools variable and the output parser (which can use OpenAI function calling parser)
- Create the agent loop
- Wrap everything into the AgentExecutor
- Invoke the agent with some query input


In [15]:
# setup the llm
from langchain_openai import ChatOpenAI

llm_chat = ChatOpenAI(temperature=0)

Now, let's create a tool for the LLM

In [17]:
import subprocess

def github_commit_tool(commit_message="Some commit"):
    subprocess.run(["git", "add", "."])
    subprocess.run(["git", "commit", "-m", commit_message])
    subprocess.run(["git", "push", "-u", "origin", "main"])
    
    return "Committed to Github"

Now! Before we use it with langchain, let's test it by itself.

In [18]:
github_commit_tool("Some testing commit")

Now, let's take a look at out github repository from the terminal.

![](2023-10-29-20-47-54.png)

Nice! It looks like we are good to go with this first simple tool!

Let's now make it a tool for our langchain agent by adding the @tool. 

In [19]:
from langchain.tools import tool

@tool
def github_commit_tool(commit_message="Some commit"):
    """This function uses the subprocess package to make commits to a github repo pre-defined."""
    subprocess.run(["git", "add", "."])
    subprocess.run(["git", "commit", "-m", commit_message])
    subprocess.run(["git", "push", "-u", "origin", "main"])
    
    return "Committed to Github"

tools = [github_commit_tool]

We added some documentation to our function to abide by the requirements of the tool decorator from langchain.

Now, let's test if a simple agent can use that tool!

Let's start by setting up our prompt template.

In [20]:
# from langchain.agents import AgentExecutor
# from langchain.prompts import ChatPromptTemplate

# template = """
# You are a helpful programming assistant. The user will ask you to perform commits to a github repo and you will perform the needed actions and call the necessary
# functions when needed.
# Your current task:
# {input}
# """
# prompt = ChatPromptTemplate.from_template(template)

In [21]:
from langchain import hub

prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

KeyboardInterrupt: 

In [ ]:
from langchain.agents import create_openai_functions_agent
from langchain_openai import ChatOpenAI

llm_chat = ChatOpenAI(temperature=0)
agent = create_openai_functions_agent(llm_chat, tools, prompt)

In [ ]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent,tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input": "Create a github commit with the message 'Pancakes are better than waffles!'"})



> Entering new AgentExecutor chain...

Invoking: `github_commit_tool` with `{'commit_message': 'Pancakes are better than waffles!'}`


[main c657b82] Pancakes are better than waffles!
 1 file changed, 19 insertions(+), 59 deletions(-)


To https://github.com/EnkrateiaLucca/oreilly_live_training_agents.git
   d18237d..c657b82  main -> main


branch 'main' set up to track 'origin/main'.
Committed to GithubA commit with the message 'Pancakes are better than waffles!' has been created on Github.

> Finished chain.


{'input': "Create a github commit with the message 'Pancakes are better than waffles!'",
 'output': "A commit with the message 'Pancakes are better than waffles!' has been created on Github."}

And voila, the agent calls the right function and creates the commit!

(ignore the github related error, let's focus on the agent capabilities).

And voila! We made a commit!

![](2023-10-30-13-04-20.png)

Ok, this is cool but, we can go beyond that, and add the ability to create a file and then commit that file to our desired repository. Let's do that!

To do it all we have to do is amplify the Agent's toolkit with another tool that can create files in the current folder.

Perfect! Now, let's add this to our github agent

In [ ]:
@tool
def create_file_tool(file_contents):
    """This function creates a file with the contents and file path provided as inputs."""
    with open("file.py", "w") as f:
        f.write(file_contents)
    
    return "File created"

Now, let's add that functionality to our agent by just updating the tools list.

In [ ]:
tools = [github_commit_tool, create_file_tool]

Now, let's modify our agent with its new toolkit

In [ ]:
agent = create_openai_functions_agent(llm_chat, tools, prompt)

In [ ]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

Now, let's try using both tools in a call to see if the agent can do something slightly more complex now.

In [ ]:
agent_executor.invoke({
    "input": "Write Python code with a for loop stating the truth about what is better: pancakes or waffles.\
    Output ONLY THE CODE and save that to a file then commit it to our github repository."
})



> Entering new AgentExecutor chain...

Invoking: `create_file_tool` with `{'file_contents': "pancakes = ['Delicious', 'Fluffy', 'Versatile']\n\nwaffles = ['Crispy', 'Golden', 'Syrup pockets']\n\nfor i in range(len(pancakes)):\n    print(f'Pancakes are {pancakes[i]}')\n    print(f'Waffles are {waffles[i]}')\n    print()  # Add a blank line for readability"}`


File created
Invoking: `github_commit_tool` with `{'commit_message': 'Add code to compare pancakes and waffles'}`


[main 144b163] Add code to compare pancakes and waffles
 2 files changed, 50 insertions(+), 6 deletions(-)
 create mode 100644 notebooks/file.py


To https://github.com/EnkrateiaLucca/oreilly_live_training_agents.git
   c657b82..144b163  main -> main


branch 'main' set up to track 'origin/main'.
Committed to GithubI have written the Python code to compare pancakes and waffles and saved it to a file. The code has been committed to our GitHub repository.

> Finished chain.


{'input': 'Write Python code with a for loop stating the truth about what is better: pancakes or waffles.    Output ONLY THE CODE and save that to a file then commit it to our github repository.',
 'output': 'I have written the Python code to compare pancakes and waffles and saved it to a file. The code has been committed to our GitHub repository.'}

And voila again!
![](2023-10-30-13-15-38.png)

For The Practice Demo We'll stop here, below I show how to expand this for multi-input tools.

Ok! This is awesome right? Let's just stop for a second and appreciate the simplicity of this implementation and the power that it gives us! Now, on the other hand, we are using a [ZeroShotReactAgent](https://python.langchain.com/docs/modules/agents/agent_types/react#:~:text=Using-,zeroshotreactagent), which is limited when it comes to multi input functions as tools. However, we can easily expand this by leveraging Langchain's amazing [Multi-Input Tools](https://python.langchain.com/docs/modules/agents/tools/multi_input_tool#:~:text=On%20this%20page-,Multi-Input%20Tools,with%20the%20StructuredTool%20class.,-import%20os).

Let's do that to modify our current tool: `create_file_tool()` to accept the file's contents as well as a file path, this way we are not restricted in creating just one type of file with a pre-specified name.

In [ ]:
from langchain.tools import StructuredTool

def create_file_tool(file_contents: str, file_path: str) -> str:
    """
    Function that creates a file with the contents 
    and file path provided as inputs.
    """
    with open(file_path, "w") as f:
        f.write(file_contents)
    
    return "File created"

# Let's test it first!
create_file_tool("print('Hello World')", "print_file.py")

'File created'

In [ ]:
!ls *.py

file.py                            print_file.py
function_calls_chatgpt.py          sample_app_to_test_github_agent.py


In [ ]:
!python print_file.py

Hello World


Great! Now, let's make it a tool for the agent.

In [ ]:
create_file_tool = StructuredTool.from_function(create_file_tool)

And, again, let's update our toolkit and our agent.

In [ ]:
tools = [github_commit_tool, create_file_tool]

In [ ]:
agent = create_openai_functions_agent(llm_chat, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

Now, let's test it by asking it to write a simple `README.md` file, giving the agent some context about the project's repo.

In [ ]:
agent_executor.invoke({
    "input": "Write a report on why pancakes are so much better than all the other options (including sarcastially waffles) for a good happy breakfast. Save that to pancakes_for_the_win.md and commit it to our github repository."
})



> Entering new AgentExecutor chain...

Invoking: `create_file_tool` with `{'file_contents': "Pancakes are the ultimate breakfast option. They are fluffy, delicious, and versatile. Unlike other breakfast options, pancakes have a unique ability to bring joy and happiness to our mornings.\n\nFirstly, let's talk about the texture. Pancakes are soft and fluffy, with a slight crispiness on the edges. The combination of the fluffy interior and the crispy exterior creates a delightful mouthfeel that is unmatched by any other breakfast option. The moment you take a bite into a pancake, you are greeted with a burst of heavenly goodness.\n\nSecondly, pancakes are incredibly versatile. You can customize them to suit your taste preferences. Whether you like them plain, with chocolate chips, or with fresh fruits, pancakes can be easily adapted to satisfy your cravings. You can also experiment with different toppings like maple syrup, honey, or whipped cream to enhance the flavor even further.\n\nN

To https://github.com/EnkrateiaLucca/oreilly_live_training_agents.git
   144b163..5108517  main -> main


branch 'main' set up to track 'origin/main'.
Committed to GithubI have created a report titled "pancakes_for_the_win.md" which explains why pancakes are the ultimate breakfast option. I have also committed the file to our GitHub repository.

> Finished chain.


{'input': 'Write a report on why pancakes are so much better than all the other options (including sarcastially waffles) for a good happy breakfast. Save that to pancakes_for_the_win.md and commit it to our github repository.',
 'output': 'I have created a report titled "pancakes_for_the_win.md" which explains why pancakes are the ultimate breakfast option. I have also committed the file to our GitHub repository.'}

And again we succeed!

![](2023-10-30-13-26-32.png)

Ok, before we finish with this example, let's add a last tool to read files from a repo, for situations where we might write some Python code and we want our agent to integrate that Python code into our repository. 

Again, we follow the same procedure:

- Create the tool
- Update our tookit
- Update our agent
- Run a test task to check if it works 

In [ ]:
def read_file_tool(file_path):
    with open(file_path, "r") as f:
        contents = f.read()
    
    return contents

read_file_tool("./pancakes_for_the_win.md")

"Pancakes are the ultimate breakfast option. They are fluffy, delicious, and versatile. Unlike other breakfast options, pancakes have a unique ability to bring joy and happiness to our mornings.\n\nFirstly, let's talk about the texture. Pancakes are soft and fluffy, with a slight crispiness on the edges. The combination of the fluffy interior and the crispy exterior creates a delightful mouthfeel that is unmatched by any other breakfast option. The moment you take a bite into a pancake, you are greeted with a burst of heavenly goodness.\n\nSecondly, pancakes are incredibly versatile. You can customize them to suit your taste preferences. Whether you like them plain, with chocolate chips, or with fresh fruits, pancakes can be easily adapted to satisfy your cravings. You can also experiment with different toppings like maple syrup, honey, or whipped cream to enhance the flavor even further.\n\nNow, let's address the elephant in the room - waffles. While some may argue that waffles are a 

Ok, our function seems to work so let's add it to our toolkit, since its a simple function with only one input we can just add the @tool decorator on top of it.

In [ ]:
@tool
def read_file_tool(file_path):
    """
    Simple function to read contents from a file.
    """
    with open(file_path, "r") as f:
        contents = f.read()
    
    return contents

And we update our toolkit:

In [ ]:
tools = [github_commit_tool, create_file_tool, read_file_tool]

And we update our agent

In [ ]:
agent = create_openai_functions_agent(llm_chat, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

Before we run our newly updated agent, let's create a .txt file containing the contents from this notebook so that our agent has something useful to read.

In [ ]:
agent_executor.invoke(
    {
    "input": "Write a simple essay that counters all the arguments in the\
        pancakes_for_the_win.md file arguing for waffles, write that to waffles_for_the_win.md and commit it to our github repository."
    }
                      )



> Entering new AgentExecutor chain...

Invoking: `read_file_tool` with `{'file_path': 'pancakes_for_the_win.md'}`


Pancakes are the ultimate breakfast option. They are fluffy, delicious, and versatile. Unlike other breakfast options, pancakes have a unique ability to bring joy and happiness to our mornings.

Firstly, let's talk about the texture. Pancakes are soft and fluffy, with a slight crispiness on the edges. The combination of the fluffy interior and the crispy exterior creates a delightful mouthfeel that is unmatched by any other breakfast option. The moment you take a bite into a pancake, you are greeted with a burst of heavenly goodness.

Secondly, pancakes are incredibly versatile. You can customize them to suit your taste preferences. Whether you like them plain, with chocolate chips, or with fresh fruits, pancakes can be easily adapted to satisfy your cravings. You can also experiment with different toppings like maple syrup, honey, or whipped cream to enhance the flavor

To https://github.com/EnkrateiaLucca/oreilly_live_training_agents.git
   5108517..c57a8cd  main -> main


branch 'main' set up to track 'origin/main'.
Committed to GithubI have written a simple essay countering all the arguments in the "pancakes_for_the_win.md" file arguing for waffles. The essay has been saved as "waffles_for_the_win.md" and committed to our GitHub repository with the commit message "Counter arguments for waffles".

> Finished chain.


{'input': 'Write a simple essay that counters all the arguments in the        pancakes_for_the_win.md file arguing for waffles, write that to waffles_for_the_win.md and commit it to our github repository.',
 'output': 'I have written a simple essay countering all the arguments in the "pancakes_for_the_win.md" file arguing for waffles. The essay has been saved as "waffles_for_the_win.md" and committed to our GitHub repository with the commit message "Counter arguments for waffles".'}

In [ ]:
!cat summary_tutorial.md

# Summary of Tutorial

This tutorial provides a step-by-step guide on how to use the GitHub agent to clone a repository, create a new branch, make changes, commit and push the changes, create a pull request, and review and merge the pull request.

![](2023-10-30-13-42-25.png)

I think this is a great usecase for agents like these even if sometimes the performance and reliability is a bit off, because now, its a matter of optimizing the edges to get it to work for complex use cases.